In [1]:
import pandas as pd

pd.set_option("display.max_rows", 999)

In [2]:
stats = pd.read_csv("/Users/camen/Downloads/all-stats-regional-2021.csv/all-stats-regional-2021.csv")

stats = stats.loc[
    (
        (stats.region_code.str.startswith("USA-")) &
        (stats.region_type == "state") &
        (stats.version_year == 2021)
    ), :
]


In [3]:
stats["common_name"] = stats["common_name"].replace("Common/Short-billed Gull", "Short-Billed Gull")
stats["common_name"] = stats["common_name"].replace("Broad-billed/Turquoise-crowned Hummingbird", "Turquoise-crowned Hummingbird")
stats["common_name"] = stats["common_name"].replace("Rufous-capped/Chestnut-capped Warbler", "Chestnut-capped Warbler")
stats["programmatic_name"] = stats.common_name.str.replace(r"[-| ]{1}", "_", regex=True).str.replace("'", "").str.lower()

In [4]:
stats.head()

,species_code,taxon_order,common_name,scientific_name,version_year,region_type,region_code,region_name,region_area_km2,season_name,start_dt,end_dt,abundance_mean,total_pop_percent,range_percent_occupied,range_total_percent,range_days_occupation,programmatic_name
733,wfwduc1,219,White-faced Whistling-Duck,Dendrocygna viduata,2021,state,USA-FL,Florida,146719.891767,year_round,2020-01-04,2020-12-28,0.0000,0.0000,0.0005,0.0000,0.0,white_faced_whistling_duck
1871,bbwduc,221,Black-bellied Whistling-Duck,Dendrocygna autumnalis,2021,state,USA-AL,Alabama,133835.049963,breeding,2020-05-24,2020-08-03,0.0148,0.0003,0.0150,0.0005,77.0,black_bellied_whistling_duck
1872,bbwduc,221,Black-bellied Whistling-Duck,Dendrocygna autumnalis,2021,state,USA-AL,Alabama,133835.049963,nonbreeding,2020-01-18,2020-03-01,0.0004,0.0000,0.0009,0.0000,0.0,black_bellied_whistling_duck
1873,bbwduc,221,Black-bellied Whistling-Duck,Dendrocygna autumnalis,2021,state,USA-AL,Alabama,133835.049963,postbreeding_migration,2020-08-10,2020-01-11,0.0056,0.0001,0.0120,0.0003,105.0,black_bellied_whistling_duck
1874,bbwduc,221,Black-bellied Whistling-Duck,Dendrocygna autumnalis,2021,state,USA-AL,Alabama,133835.049963,prebreeding_migration,2020-03-08,2020-05-17,0.0031,0.0000,0.0086,0.0003,63.0,black_bellied_whistling_duck


In [5]:
stats = stats.groupby(by=["common_name", "scientific_name", "region_code", "programmatic_name"])["abundance_mean"].max().reset_index().set_index("programmatic_name", inplace=False)

In [6]:
import glob

In [7]:
all_bird_filepaths = {}
for fp in glob.glob("/Users/camen/Downloads/birdsnap_500/download/images/*/*.jpg"):
    bird_name = fp.split("/")[-2].lower()
    bird_filepaths = all_bird_filepaths.get(bird_name, [])
    bird_filepaths.append(fp)
    all_bird_filepaths[bird_name] = bird_filepaths
    
for fp in glob.glob("/Users/camen/Downloads/caltech_ucsd_birds_200_images/*/*.jpg"):
    bird_name = fp.split("/")[-2].split(".")[-1].lower()
    bird_filepaths = all_bird_filepaths.get(bird_name, [])
    bird_filepaths.append(fp)
    all_bird_filepaths[bird_name] = bird_filepaths
    
for fp in set(
    glob.glob("/Users/camen/Downloads/kaggle_birds_400/train/*/*.jpg") +
    glob.glob("/Users/camen/Downloads/kaggle_birds_400/test/*/*.jpg") + 
    glob.glob("/Users/camen/Downloads/kaggle_birds_400/valid/*/*.jpg")
):
    bird_name = fp.split("/")[-2].lower().replace(" ", "_").replace("&", "and")
    bird_filepaths = all_bird_filepaths.get(bird_name, [])
    bird_filepaths.append(fp)
    all_bird_filepaths[bird_name] = bird_filepaths

In [8]:
birds = stats.copy()
birds["images"] = pd.Series(all_bird_filepaths)
birds = birds.loc[~birds.images.isna(), :].copy()
birds["num_images"] = birds.images.apply(len)
birds.head()

,common_name,scientific_name,region_code,abundance_mean,images,num_images
programmatic_name,,,,,,
acadian_flycatcher,Acadian Flycatcher,Empidonax virescens,USA-AL,0.3790,[/Users/camen/Downloads/birdsnap_500/download/...,104
acadian_flycatcher,Acadian Flycatcher,Empidonax virescens,USA-AR,0.5655,[/Users/camen/Downloads/birdsnap_500/download/...,104
acadian_flycatcher,Acadian Flycatcher,Empidonax virescens,USA-CT,0.1247,[/Users/camen/Downloads/birdsnap_500/download/...,104
acadian_flycatcher,Acadian Flycatcher,Empidonax virescens,USA-DC,0.5859,[/Users/camen/Downloads/birdsnap_500/download/...,104
acadian_flycatcher,Acadian Flycatcher,Empidonax virescens,USA-DE,0.2973,[/Users/camen/Downloads/birdsnap_500/download/...,104


In [9]:
massachusetts = birds[birds.region_code == "USA-MA"].copy().sort_values(by="abundance_mean", ascending=False)

In [10]:
massachusetts.head()

,common_name,scientific_name,region_code,abundance_mean,images,num_images
programmatic_name,,,,,,
canada_goose,Canada Goose,Branta canadensis,USA-MA,11.9505,[/Users/camen/Downloads/birdsnap_500/download/...,81
american_robin,American Robin,Turdus migratorius,USA-MA,10.1348,[/Users/camen/Downloads/birdsnap_500/download/...,92
common_grackle,Common Grackle,Quiscalus quiscula,USA-MA,9.1693,[/Users/camen/Downloads/birdsnap_500/download/...,280
common_eider,Common Eider,Somateria mollissima,USA-MA,8.6867,[/Users/camen/Downloads/birdsnap_500/download/...,89
european_starling,European Starling,Sturnus vulgaris,USA-MA,7.9318,[/Users/camen/Downloads/birdsnap_500/download/...,87


In [11]:
import numpy as np


def sample(df, N, state=None, most_common=False):
     
    df_sampled = birds.copy().sort_values(by="abundance_mean", ascending=False)
    if state is not None:
        df_sampled = df_sampled.loc[df_sampled.region_code == f"USA-{state.upper()}"]
    if most_common:
        df_sampled = df_sampled.iloc[:N]
    else:
        df_sampled = df_sampled.sample(N, replace=False)

    df_sampled["image"] = df_sampled.images.apply(np.random.choice)
    return df_sampled[["common_name", "scientific_name", "region_code", "abundance_mean", "image"]]


def get(df, birds):
    birds = [bird.lower().replace("'", "").replace(r"[- ]{1}", "_") for bird in birds]
    df_sampled = df.copy().loc[birds, :].groupby(level=0).first()
    df_sampled["image"] = df_sampled.images.apply(np.random.choice)
    return df_sampled[["common_name", "scientific_name", "abundance_mean", "image"]]

In [12]:
df_specific = get(df=birds, birds=["red_shouldered_hawk"])
df_specific

,common_name,scientific_name,abundance_mean,image
programmatic_name,,,,
red_shouldered_hawk,Red-shouldered Hawk,Buteo lineatus,0.4663,/Users/camen/Downloads/birdsnap_500/download/i...


In [38]:
df_sample = sample(df=birds, N=100, state="MA", most_common=True)
df_sample

,common_name,scientific_name,region_code,abundance_mean,image
programmatic_name,,,,,
canada_goose,Canada Goose,Branta canadensis,USA-MA,11.9505,/Users/camen/Downloads/birdsnap_500/download/i...
american_robin,American Robin,Turdus migratorius,USA-MA,10.1348,/Users/camen/Downloads/birdsnap_500/download/i...
common_grackle,Common Grackle,Quiscalus quiscula,USA-MA,9.1693,/Users/camen/Downloads/kaggle_birds_400/train/...
common_eider,Common Eider,Somateria mollissima,USA-MA,8.6867,/Users/camen/Downloads/birdsnap_500/download/i...
european_starling,European Starling,Sturnus vulgaris,USA-MA,7.9318,/Users/camen/Downloads/birdsnap_500/download/i...
common_tern,Common Tern,Sterna hirundo,USA-MA,7.5709,/Users/camen/Downloads/caltech_ucsd_birds_200_...
tree_swallow,Tree Swallow,Tachycineta bicolor,USA-MA,7.4150,/Users/camen/Downloads/kaggle_birds_400/train/...
red_winged_blackbird,Red-winged Blackbird,Agelaius phoeniceus,USA-MA,6.1499,/Users/camen/Downloads/kaggle_birds_400/train/...
black_capped_chickadee,Black-capped Chickadee,Poecile atricapillus,USA-MA,5.9909,/Users/camen/Downloads/birdsnap_500/download/i...


In [15]:
# birds.reset_index().to_csv("/Users/camen/Documents/bird-herd/backend/api/resources/birds.csv.gz", index=False)

In [16]:
# birds.reset_index().sample(20).to_csv("/Users/camen/Documents/bird-herd/backend/tests/resources/birds.csv.gz", index=False)